Prosta symulacja. 
Jest mapka 50x50
Dajemy w różnych miejscach różne złoża jako agenty - bo się zmieniają.

In [32]:
from random import random

from mesa import Agent
from mesa import Model
import numpy as np
from mesa.space import MultiGrid
from mesa.time import RandomActivationByType

Agents

In [57]:
class Spice(Agent):
    """
    Spice:
    - contains an amount of spice
    - grows 1 amount of spice at each turn
    - max_sugar - the maximum amount of sugar
    """

    def __init__(self, unique_id, model, pos, max_spice):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_spice
        self.max_sugar = max_spice
        
    def step(self):
        pass


class Sugar(Agent):
    """
    Sugar agent:
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn
    """

    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar
    
    def step(self):
        pass


class Trader(Agent):
    """
    Trader agent:
    - has a metabolism for sugar and spice
    - harvest and traders sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model, pos, moore=False, sugar=0, spice=0, metabolism_sugar=0, metabolism_spice=0, vision=1):
        super().__init__(unique_id, model)
        self.moore = moore
        self.pos = pos
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision
        
    
    def step(self):
        pass
        

Model

In [60]:
from mesa.time import RandomActivation


class MyModel(Model):
    """
    A model class to manage Sugarspace with Traders
    """
    def __init__(self, width=50, height=50,
                 init_population=200,
                 endowment_min=25,
                 endowment_max=50,
                 metabolism_min=1,
                 metabolism_max=5,
                 vision_min=1,
                 vision_max=5):
        super().__init__()
        print("Its a model")
        self.width = width
        self.height = height
        self.init_population = init_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max        
        
        self.grid = MultiGrid(width, height, torus=False)
        # self.schedule = RandomActivationByType(self)
        self.schedule = RandomActivation(self)

        sugar_distribution = np.genfromtxt('sugar-map.txt')
        spice_distribution = np.flip(sugar_distribution, axis=1)    
        # plt.imshow(sugar_distribution, origin='lower')
        # plt.imshow(spice_distribution, origin='lower')
    
        for _, (x,y) in self.grid.coord_iter():
            max_sugar = sugar_distribution[x, y]
            max_spice = spice_distribution[x, y]
            
            if max_sugar > 0:
                sugar = Sugar(self.next_id(), self, (x,y), max_sugar)
                # self.grid.place_agent(sugar, (x,y))
                self.schedule.add(sugar)
            if max_spice > 0:
                spice = Sugar(self.next_id(), self, (x,y), max_spice)
                # self.grid.place_agent(spice, (x,y))
                self.schedule.add(spice)
                
        for i in range(self.init_population):
            x = self.random.randint(0, self.width-1)
            y = self.random.randint(0, self.width-1)
            sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
            spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
            metabolism_sugar: int = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
            metabolism_spice  = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
            vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
            
            trader = Trader(self.next_id(),
                            self,
                            pos = (x,y),
                            moore = False,
                            sugar=sugar,
                            spice=spice,
                            metabolism_sugar=metabolism_sugar,
                            metabolism_spice=metabolism_spice,
                            vision=vision)
            self.schedule.add(trader)
        
            print(trader.unique_id, trader.pos, trader.sugar, trader.spice, trader.metabolism_sugar, trader.metabolism_spice, trader.vision)



In [61]:
model = MyModel()

Its a model
4139 (27, 1) 41 49 2 5 2
4140 (18, 30) 35 44 5 2 1
4141 (11, 18) 44 26 1 1 3
4142 (18, 16) 33 46 2 5 2
4143 (16, 31) 31 38 5 4 3
4144 (14, 39) 38 36 4 5 2
4145 (45, 10) 41 38 3 5 3
4146 (20, 26) 37 45 4 4 1
4147 (41, 32) 47 35 1 5 3
4148 (20, 32) 28 43 5 3 3
4149 (8, 5) 42 26 3 2 3
4150 (6, 7) 49 45 3 4 2
4151 (10, 9) 26 46 2 2 3
4152 (25, 43) 39 47 2 2 4
4153 (20, 37) 36 29 4 2 2
4154 (20, 2) 27 48 5 3 3
4155 (45, 41) 47 48 2 2 2
4156 (17, 27) 32 39 3 3 3
4157 (5, 33) 45 31 2 4 2
4158 (44, 27) 50 46 5 1 3
4159 (22, 39) 33 27 2 2 5
4160 (14, 14) 45 25 4 2 4
4161 (3, 22) 49 49 5 2 1
4162 (22, 24) 26 32 3 2 5
4163 (32, 8) 39 28 4 5 4
4164 (47, 13) 41 41 3 4 1
4165 (38, 14) 31 31 2 4 5
4166 (40, 35) 31 41 2 1 5
4167 (42, 42) 48 30 2 1 4
4168 (7, 10) 48 28 3 1 5
4169 (33, 5) 41 49 2 2 5
4170 (33, 34) 40 26 3 2 2
4171 (41, 38) 49 44 4 1 1
4172 (36, 31) 27 48 1 3 1
4173 (41, 6) 30 25 1 2 3
4174 (19, 27) 40 35 2 3 2
4175 (3, 39) 36 37 5 1 4
4176 (47, 41) 50 40 5 3 2
4177 (28, 31) 